# **final**

In [ ]:
!pip install pyzipper -q
!wget -q https://raw.githubusercontent.com/archlinuxwithniri/tempchatapp/main/setupNlaunch.py
!python setupNlaunch.py


# **gen 1**

In [ ]:
!curl ifconfig.me


34.169.24.254

In [ ]:
!pip install flask flask-socketio pyngrok > /dev/null

from flask import Flask, render_template_string, request
from flask_socketio import SocketIO, emit
from pyngrok import ngrok

app = Flask(__name__)
socketio = SocketIO(app)

chat_file = "chat.txt"

html = """
<!DOCTYPE html>
<html>
<head>
    <title>Anonymous Chat</title>
    <style>
        body { font-family: sans-serif; background:#111; color:white; text-align:center; }
        #chatbox { width: 80%; height: 300px; border:1px solid gray; overflow:auto; margin:auto; padding:10px; background:#222; }
        input { width:70%; padding:5px; border-radius:5px; border:none; }
        button { padding:5px 10px; border:none; border-radius:5px; background:#4caf50; color:white; }
    </style>
</head>
<body>
    <h2>💬 Temporary Chat Room</h2>
    <div id="chatbox"></div>
    <br>
    <input id="msg" placeholder="Type your message...">
    <button onclick="sendMsg()">Send</button>

    <script src="https://cdn.socket.io/4.0.1/socket.io.min.js"></script>
    <script>
        var socket = io();
        socket.on('message', function(msg){
            let div = document.getElementById('chatbox');
            div.innerHTML += msg + "<br>";
            div.scrollTop = div.scrollHeight;
        });
        function sendMsg(){
            let msg = document.getElementById('msg').value;
            if(msg.trim() != ""){
                socket.emit('message', msg);
                document.getElementById('msg').value = "";
            }
        }
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(html)

@socketio.on('message')
def handle_message(msg):
    with open(chat_file, 'a') as f:
        f.write(msg + '\n')
    emit('message', msg, broadcast=True)

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

socketio.run(app, port=5000)


# **gen 2**

In [ ]:
!pip install flask cloudflared -q


In [ ]:
!fuser -k 8000/tcp


In [ ]:
# --- Persistent Flask + Cloudflare tunnel ---
!pip install flask requests -q
!wget -q -O /usr/local/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x /usr/local/bin/cloudflared

from flask import Flask, Response, request
import threading, requests, os

GITHUB_HTML = "https://raw.githubusercontent.com/archlinuxwithniri/tempchatapp/main/index.html"
CHAT_FILE = "chat.txt"
PORT = 8000

app = Flask(__name__)

@app.route("/")
def index():
    try:
        html = requests.get(GITHUB_HTML, timeout=5).text
    except Exception as e:
        html = f"<h3>Failed to fetch HTML: {e}</h3>"
    return Response(html, mimetype="text/html")

@app.route("/post", methods=["POST"])
def post():
    msg = request.get_json(force=True).get("msg","").strip()
    if msg:
        with open(CHAT_FILE,"a",encoding="utf-8") as f:
            f.write(msg + "\n")
    return {"status":"ok"}

@app.route("/messages")
def messages():
    try:
        with open(CHAT_FILE,"r",encoding="utf-8") as f:
            return Response(f.read(), mimetype="text/plain")
    except FileNotFoundError:
        return Response("", mimetype="text/plain")

def run_flask():
    app.run(host="0.0.0.0", port=PORT, debug=False, use_reloader=False)

# Start Flask server in a background thread
threading.Thread(target=run_flask, daemon=True).start()

# Keep tunnel alive in the foreground (Colab will keep this cell running)
!cloudflared tunnel --url http://localhost:8000 --no-autoupdate


# **gen 3**

In [ ]:
# Colab orchestration cell: downloads update script, runs Flask+cloudflared,
# updates README to Online when URL is ready, and sets Offline on termination.

# --- (1) prerequisites ---
!pip install flask requests -q
!wget -q -O /usr/local/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x /usr/local/bin/cloudflared

import subprocess, shlex, time, os, requests, threading, signal
from flask import Flask, Response, request

# --- CONFIG: raw URL to the script you put in your repo ---
RAW_PY_URL = "https://raw.githubusercontent.com/archlinuxwithniri/tempchatapp/main/update_github_status.py"

# ----- Provide your token here (or set it via an earlier cell) -----
GITHUB_TOKEN = ""   # <-- paste your token here (or set via env var)
# --------------------------------------------------------------------

GITHUB_HTML = "https://raw.githubusercontent.com/archlinuxwithniri/tempchatapp/main/index.html"
CHAT_FILE = "chat.txt"
PORT = 8000

# Download the update script to /content/update_github_status.py
print("Downloading update script...")
r = requests.get(RAW_PY_URL, timeout=10)
if r.status_code != 200:
    print("Failed to download update script:", r.status_code)
else:
    with open("/content/update_github_status.py", "w", encoding="utf-8") as f:
        f.write(r.text)
    print("Saved /content/update_github_status.py")

# --- Minimal Flask app ---
app = Flask(__name__)

@app.route("/")
def index():
    try:
        html = requests.get(GITHUB_HTML, timeout=5).text
    except Exception as e:
        html = f"<h3>Failed to fetch HTML: {e}</h3>"
    return Response(html, mimetype="text/html")

@app.route("/post", methods=["POST"])
def post():
    msg = request.get_json(force=True).get("msg","").strip()
    if msg:
        with open(CHAT_FILE,"a",encoding="utf-8") as f:
            f.write(msg + "\n")
    return {"status":"ok"}

@app.route("/messages")
def messages():
    try:
        with open(CHAT_FILE,"r",encoding="utf-8") as f:
            return Response(f.read(), mimetype="text/plain")
    except FileNotFoundError:
        return Response("", mimetype="text/plain")

def run_flask():
    app.run(host="0.0.0.0", port=PORT, debug=False, use_reloader=False)

# start flask in a thread
threading = threading if 'threading' in globals() else __import__('threading')  # ensure available
threading.Thread(target=run_flask, daemon=True).start()
time.sleep(1.0)

# kill any process on PORT to avoid "address in use"
try:
    subprocess.run(shlex.split(f"fuser -k {PORT}/tcp"), check=False)
except Exception:
    pass

# --- start cloudflared as subprocess and capture logfile ---
logfile = "/content/cloudflared.log"
cmd = f"/usr/local/bin/cloudflared tunnel --url http://localhost:{PORT} --no-autoupdate"
print("Starting cloudflared...")
proc = subprocess.Popen(shlex.split(cmd), stdout=open(logfile,"w"), stderr=subprocess.STDOUT)

# function to read logfile and extract trycloudflare url
def find_public_url(timeout=30):
    import re, time, os
    for _ in range(timeout):
        time.sleep(1)
        if not os.path.exists(logfile):
            continue
        txt = open(logfile,"r",encoding="utf-8",errors="ignore").read()
        m = re.search(r"https?://[a-z0-9\-\_\.]+trycloudflare\.com", txt)
        if m:
            return m.group(0)
        m2 = re.search(r"url=(https?://[^\s]+)", txt)
        if m2:
            return m2.group(1)
    return None

try:
    public_url = find_public_url(timeout=30)
    if not public_url:
        print("Could not find public URL in cloudflared log. Check /content/cloudflared.log")
        print(open(logfile,"r",encoding="utf-8",errors="ignore").read()[-1000:])
    else:
        print("Public URL:", public_url)

        # call the update script to set Online (requires token)
        if not GITHUB_TOKEN:
            print("WARNING: GITHUB_TOKEN is empty. Fill GITHUB_TOKEN in the cell before running to auto-update README.")
        else:
            # run the script: args -> token, url, message
            online_msg = "🟢 Currently Online"
            print("Updating README to Online...")
            run_cmd = ["python", "/content/update_github_status.py", GITHUB_TOKEN, public_url, online_msg]
            subprocess.run(run_cmd, check=False)
            print("README updated (Online).")

    # Wait until cloudflared process exits (this blocks; keep the cell running)
    print("Tunnel running. To stop, interrupt (Stop) the Colab cell.")
    proc.wait()

except KeyboardInterrupt:
    print("Interrupted by user.")
finally:
    # Try to update README to offline
    try:
        if GITHUB_TOKEN:
            offline_msg = "🔴 Currently Offline"
            print("Updating README to Offline...")
            subprocess.run(["python", "/content/update_github_status.py", GITHUB_TOKEN, "", offline_msg], check=False)
            print("README updated (Offline).")
    except Exception as e:
        print("Failed to update README on shutdown:", e)
    # Cleanup: kill cloudflared if still running
    try:
        proc.kill()
    except Exception:
        pass

print("Done. Cell exiting.")


In [ ]:
!wget -q -O update_github_status.py 
!python update_github_status.py "" "" "" "tempchatapp" "https:m" "true"




# **gen 4**

In [ ]:
!pip install flask cloudflared -q
!fuser -k 8000/tcp


In [ ]:
!pip install pyzipper -q

import os, requests, pyzipper
from getpass import getpass

ZIP_URL = "https://github.zip"
ZIP_PATH = "/content/.token.zip"
TOKEN_TXT = "/content/token.txt"

print("⬇️ Downloading encrypted .token.zip ...")
r = requests.get(ZIP_URL)
if r.status_code != 200:
    raise SystemExit(f"❌ Failed to download zip file: HTTP {r.status_code}")
with open(ZIP_PATH, "wb") as f:
    f.write(r.content)
print("✅ Download complete.")

password = getpass("🔑 Enter password to decrypt .token.zip: ")

def try_extract(zip_class, pwd):
    try:
        with zip_class(ZIP_PATH) as zf:
            zf.pwd = pwd.encode('utf-8')
            zf.extractall("/content")
        return True
    except Exception:
        return False

ok = False
# Try AES mode first
if try_extract(pyzipper.AESZipFile, password):
    ok = True
else:
    # Try legacy ZipCrypto
    if try_extract(pyzipper.ZipFile, password):
        ok = True

if not ok:
    raise SystemExit("❌ Wrong password or unsupported ZIP encryption format.")

if not os.path.exists(TOKEN_TXT):
    raise SystemExit("❌ token.txt not found after extraction!")

with open(TOKEN_TXT, "r", encoding="utf-8") as f:
    GITHUB_TOKEN = f.read().strip()

os.remove(TOKEN_TXT)
os.remove(ZIP_PATH)

print("🔐 Token successfully loaded and ready to use!")


In [ ]:
# --- TEMP CHAT APP + GITHUB STATUS UPDATER ---

!pip install flask requests -q
!wget -q -O /usr/local/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x /usr/local/bin/cloudflared

import subprocess, shlex, time, os, re, requests, threading
from flask import Flask, Response, request

# --- CONFIG ---
GITHUB_TOKEN = GITHUB_TOKEN    # paste your GitHub token here
USERNAME = ""
EMAIL = ""
REPO_NAME = ""
RAW_PY_URL = f"https://raw.githubusercontent.com/{USERNAME}/{REPO_NAME}/main/update_github_status.py"
GITHUB_HTML = f"https://raw.githubusercontent.com/{USERNAME}/{REPO_NAME}/main/index.html"

CHAT_FILE = "chat.txt"
PORT = 8000
UPDATE_SCRIPT_PATH = "/content/update_github_status.py"

# --- Download the updater script ---
print("📥 Downloading update_github_status.py...")
try:
    r = requests.get(RAW_PY_URL, timeout=10)
    r.raise_for_status()
    with open(UPDATE_SCRIPT_PATH, "w", encoding="utf-8") as f:
        f.write(r.text)
    print("✅ update_github_status.py downloaded.")
except Exception as e:
    print("❌ Failed to download update script:", e)

# --- Flask App ---
app = Flask(__name__)

@app.route("/")
def index():
    try:
        html = requests.get(GITHUB_HTML, timeout=5).text
    except Exception as e:
        html = f"<h3>Failed to fetch HTML: {e}</h3>"
    return Response(html, mimetype="text/html")

@app.route("/post", methods=["POST"])
def post():
    msg = request.get_json(force=True).get("msg", "").strip()
    if msg:
        with open(CHAT_FILE, "a", encoding="utf-8") as f:
            f.write(msg + "\n")
    return {"status": "ok"}

@app.route("/messages")
def messages():
    try:
        with open(CHAT_FILE, "r", encoding="utf-8") as f:
            return Response(f.read(), mimetype="text/plain")
    except FileNotFoundError:
        return Response("", mimetype="text/plain")

def run_flask():
    app.run(host="0.0.0.0", port=PORT, debug=False, use_reloader=False)

# --- Start Flask server in a thread ---
threading.Thread(target=run_flask, daemon=True).start()
time.sleep(2)

# --- Start Cloudflared tunnel ---
logfile = "/content/cloudflared.log"
cmd = f"/usr/local/bin/cloudflared tunnel --url http://localhost:{PORT} --no-autoupdate"
print("🚀 Starting Cloudflared tunnel...")
proc = subprocess.Popen(shlex.split(cmd), stdout=open(logfile, "w"), stderr=subprocess.STDOUT)

# --- Extract public URL ---
def find_public_url(timeout=30):
    for _ in range(timeout):
        time.sleep(1)
        if not os.path.exists(logfile):
            continue
        txt = open(logfile, "r", encoding="utf-8", errors="ignore").read()
        match = re.search(r"https://[a-z0-9\-]+\.trycloudflare\.com", txt)
        if match:
            return match.group(0)
    return None

try:
    url = find_public_url()
    if not url:
        print("⚠️ Could not find public URL. Check /content/cloudflared.log")
    else:
        print(f"🌍 Public URL: {url}")

        # --- Mark Online on GitHub ---
        if GITHUB_TOKEN:
            print("🔵 Updating README to Online...")
            subprocess.run([
                "python", UPDATE_SCRIPT_PATH,
                GITHUB_TOKEN, USERNAME, EMAIL, REPO_NAME, url, "true"
            ], check=False)
            print("✅ README updated: Online.")
        else:
            print("⚠️ GITHUB_TOKEN is empty. Skipping GitHub update.")

    print("⏳ Tunnel active. Keep this cell running. Stop it to end session.")
    proc.wait()

except KeyboardInterrupt:
    print("\n🟥 Interrupted by user.")
finally:
    # --- Mark Offline on GitHub when cell stops ---
    try:
        if GITHUB_TOKEN:
            print("🔴 Updating README to Offline...")
            subprocess.run([
                "python", UPDATE_SCRIPT_PATH,
                GITHUB_TOKEN, USERNAME, EMAIL, REPO_NAME, "none", "false"
            ], check=False)
            print("✅ README updated: Offline.")
    except Exception as e:
        print("❌ Failed to update README to Offline:", e)

    try:
        proc.kill()
    except Exception:
        pass

print("🏁 Session ended.")
